In [1]:
## Import packages
import os
import csv
import numpy as np
#from scipy.io import wavfile

import tensorflow as tf

flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
flags.DEFINE_string('output_tfrecords_file', 'tfrecords/added_train_data',
                    'File containing tfrecords will be written at this path.')
flags.DEFINE_string('model_dir', 'yt8m',
                    'Directory to store model files.')

In [5]:
# The following flags are set to match the YouTube-8M dataset format.
flags.DEFINE_integer('frames_per_second', 1,
                   'This many frames per second will be processed')
flags.DEFINE_string('labels_feature_key', 'labels',
                  'Labels will be written to context feature with this '
                  'key, as int64 list feature.')
flags.DEFINE_string('image_feature_key', 'rgb',
                  'Image features will be written to sequence feature with '
                  'this key, as bytes list feature, with only one entry, '
                  'containing quantized feature string.')
flags.DEFINE_string('video_file_key_feature_key', 'video_id',
                  'Input <video_file> will be written to context feature '
                  'with this key, as bytes list feature, with only one '
                  'entry, containing the file path of the video. This '
                  'can be used for debugging but not for training or eval.')
flags.DEFINE_boolean('insert_zero_audio_features', True,
                   'If set, inserts features with name "audio" to be 128-D '
                   'zero vectors. This allows you to use YouTube-8M '
                   'pre-trained model.')

In [22]:
## Stap 1:
## Download de volgende files en zet ze in model_dir (bv /yt8m)
#'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
#'http://data.yt8m.org/yt8m_pca.tgz'

In [33]:
# load inception
inception_proto_file = os.path.join(model_dir, 'classify_image_graph_def.pb')
graph_def = tf.GraphDef.FromString(open(inception_proto_file, 'rb').read())
inception_graph = tf.Graph()
with inception_graph.as_default():
    _ = tf.import_graph_def(graph_def, name='')
    session = tf.Session()

In [40]:
# load pca
pca_mean = np.load(
    os.path.join(model_dir, 'mean.npy'))[:, 0]
pca_eigenvals = np.load(
    os.path.join(model_dir, 'eigenvals.npy'))[:1024, 0]
pca_eigenvecs = np.load(
    os.path.join(model_dir, 'eigenvecs.npy')).T[:, :1024]

In [43]:
writer = tf.python_io.TFRecordWriter(FLAGS.output_tfrecords_file)

In [27]:
total_written = 0
total_error = 0

In [ ]:
for video_file, labels in csv.reader(open(FLAGS.input_videos_csv)):
    rgb_features = []
    for rgb in frame_iterator(video_file, every_ms=1000.0/FLAGS.frames_per_second):
        features = extractor.extract_rgb_frame_features(rgb[:, :, ::-1])
        rgb_features.append(_bytes_feature(quantize(features)))

    if not rgb_features:
        print >> sys.stderr, 'Could not get features for ' + video_file
        total_error += 1
        continue

    # Create SequenceExample proto and write to output.
    feature_list = {
        FLAGS.image_feature_key: tf.train.FeatureList(feature=rgb_features),
    }
    if FLAGS.insert_zero_audio_features:
      feature_list['audio'] = tf.train.FeatureList(
          feature=[_bytes_feature(_make_bytes([0] * 128))] * len(rgb_features))

    example = tf.train.SequenceExample(
        context=tf.train.Features(feature={
            FLAGS.labels_feature_key:
                _int64_list_feature(sorted(map(int, labels.split(';')))),
            FLAGS.video_file_key_feature_key:
                _bytes_feature(_make_bytes(map(ord, video_file))),
        }),
        feature_lists=tf.train.FeatureLists(feature_list=feature_list))
    writer.write(example.SerializeToString())
    total_written += 1

In [44]:
## In feature_extractor.py (init) zie je hoe je dat via Python-code kan doen
import csv

In [4]:
## Stap 1 en 2
## This function reads the wav file and converts the samples into np arrays of [batch size, num frames, num bands]
examples_batch = vggish_input.wavfile_to_examples(FLAGS.wav_file)
print(examples_batch.shape)

C:\Users\myrna\Anaconda3\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


(31, 96, 64)


In [5]:
## Read csv-file with labels
class_map = {}
with open(vggish_params.CLASS_LABELS_INDICES) as f:
    next(f)  # skip header
    reader = csv.reader(f)
    for row in reader:
        class_map[int(row[0])] = row[2]

In [6]:
## Stap 3
with tf.Graph().as_default(), tf.Session() as sess:
    # Define the model: load the checkpoint and locate input and output tensors
    # Input: [batch_size, num_frames, num_bands] 
    # where [num_frames, num_bands] represents log-mel-scale spectrogram
    # Output: embeddings
    vggish_slim.define_vggish_slim(training=False)
    vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)
    
    pca_params = np.load(vggish_params.VGGISH_PCA_PARAMS)
    pca_matrix = pca_params[vggish_params.PCA_EIGEN_VECTORS_NAME]
    pca_means = pca_params[vggish_params.PCA_MEANS_NAME].reshape(-1, 1)
    
    features_tensor = sess.graph.get_tensor_by_name(
        vggish_params.VGGISH_INPUT_TENSOR_NAME)
    embedding_tensor = sess.graph.get_tensor_by_name(
        vggish_params.VGGISH_OUTPUT_TENSOR_NAME)
    vggish_slim.load_youtube_model(sess, vggish_params.YOUTUBE_CHECKPOINT_FILE)
    
    # Run inference and postprocessing
    [embedding_batch] = sess.run([embedding_tensor],
                                 feed_dict={features_tensor: examples_batch})
    
    postprocessed_batch = np.dot(
            pca_matrix, (embedding_batch.T - pca_means)
        ).T
    #print(postprocessed_batch)
    
    num_frames = np.minimum(postprocessed_batch.shape[0], vggish_params.MAX_FRAMES)
    data = vggish_postprocess.resize(postprocessed_batch, 0, vggish_params.MAX_FRAMES)
    data = np.expand_dims(data, 0)
    num_frames = np.expand_dims(num_frames, 0)
    
    input_tensor = sess.graph.get_collection("input_batch_raw")[0]
    num_frames_tensor = sess.graph.get_collection("num_frames")[0]
    predictions_tensor = sess.graph.get_collection("predictions")[0]
    
    ## Stap 4
    predictions_val, = sess.run(
        [predictions_tensor],
        feed_dict={
            input_tensor: data,
            num_frames_tensor: num_frames
        })

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt


In [7]:
## Filter predictions (give top 20 where p>0.1)
count = vggish_params.PREDICTIONS_COUNT_LIMIT
hit = vggish_params.PREDICTIONS_HIT_LIMIT
top_indices = np.argpartition(predictions_val[0], -count)[-count:]
line = ((class_map[i], float(predictions_val[0][i])) for i in top_indices if predictions_val[0][i] > hit)
predictions = sorted(line, key=lambda p: -p[1])

In [8]:
print(predictions)

[('Vehicle', 0.9999919533729553), ('Music', 0.7836809158325195), ('Car', 0.7723241448402405), ('Bus', 0.7148009538650513), ('Toot', 0.4626094698905945)]
